Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.

from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [13]:
graph2 = tf.Graph()
train_subset = 10000
with graph2.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  
  hidden = 1024
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden ]))
  bias1 = tf.Variable(tf.zeros([hidden ]))
  
  #weights1 = tf.Variable(
  #  tf.truncated_normal([image_size * image_size, num_labels]))
  #bias1 = tf.Variable(tf.zeros([num_labels]))
  
  
  weights2 = tf.Variable(
    tf.truncated_normal([hidden , num_labels ]))
  bias2 = tf.Variable(tf.zeros([num_labels]))

  
  def get_logits(input, weights1, bias1, weights2, bias2):
      logits1 = tf.matmul(input, weights1) + bias1
      
      logits2 = tf.nn.tanh(logits1)
      logits = tf.matmul(logits2,weights2 ) + bias2
      
      return logits

       
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits_train = get_logits(  tf_train_dataset, weights1, bias1, weights2, bias2)
  loss = (tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_train)) + \
    0.01*tf.nn.l2_loss(weights1) + \
    0.01*tf.nn.l2_loss(weights2))
  
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits_train)
  valid_prediction = tf.nn.softmax(get_logits(  tf_valid_dataset, weights1, bias1, weights2, bias2))
  test_prediction =  tf.nn.softmax(get_logits(  tf_test_dataset, weights1, bias1, weights2, bias2))
  

In [14]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph2) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [ ]:
graph3 = tf.Graph()
train_subset = 100
with graph3.as_default():
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    
    hidden = 1024
    weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden ]))
    bias1 = tf.Variable(tf.zeros([hidden ]))
    
    #weights1 = tf.Variable(
    #  tf.truncated_normal([image_size * image_size, num_labels]))
    #bias1 = tf.Variable(tf.zeros([num_labels]))
    
    
    weights2 = tf.Variable(tf.truncated_normal([hidden , num_labels ]))
    bias2 = tf.Variable(tf.zeros([num_labels]))
    
    
    def get_logits(input, weights1, bias1, weights2, bias2):
        logits1 = tf.matmul(input, weights1) + bias1
        
        logits2 = tf.nn.tanh(logits1)
        logits = tf.matmul(logits2,weights2 ) + bias2
        
        return logits
    
       
    
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits_train = get_logits(  tf_train_dataset, weights1, bias1, weights2, bias2)
    loss = (tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_train)) + \
    0.01*tf.nn.l2_loss(weights1) + \
    0.01*tf.nn.l2_loss(weights2))
    
    
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits_train)
    valid_prediction = tf.nn.softmax(get_logits(  tf_valid_dataset, weights1, bias1, weights2, bias2))
    test_prediction =  tf.nn.softmax(get_logits(  tf_test_dataset, weights1, bias1, weights2, bias2))
    
    


In [14]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph3) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 3185.534668
Training accuracy: 0.0%
Validation accuracy: 0.0%
Loss at step 100: 1175.666138
Training accuracy: 0.0%
Validation accuracy: 0.0%


/home/diegoami/anaconda3/envs/sandbox/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  """


Loss at step 200: 482.081848
Training accuracy: 0.0%
Validation accuracy: 0.0%
Loss at step 300: 264.550781
Training accuracy: 0.0%
Validation accuracy: 0.0%


Loss at step 400: 212.677979
Training accuracy: 0.0%
Validation accuracy: 0.0%
Loss at step 500: 213.076340
Training accuracy: 0.0%
Validation accuracy: 0.0%


Loss at step 600: 228.753387
Training accuracy: 0.0%
Validation accuracy: 0.0%
Loss at step 700: 246.550919
Training accuracy: 0.0%
Validation accuracy: 0.0%


Loss at step 800: 254.864075
Training accuracy: 0.0%
Validation accuracy: 0.0%
Test accuracy: 0.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [5]:
graph4 = tf.Graph()
train_subset = 10000
with graph4.as_default():
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    
    hidden = 1024
    weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden ]))
    bias1 = tf.Variable(tf.zeros([hidden ]))
    
    #weights1 = tf.Variable(
    #  tf.truncated_normal([image_size * image_size, num_labels]))
    #bias1 = tf.Variable(tf.zeros([num_labels]))
    
    
    weights2 = tf.Variable(tf.truncated_normal([hidden , num_labels ]))
    bias2 = tf.Variable(tf.zeros([num_labels]))
    
    
    def get_logits(input, weights1, bias1, weights2, bias2, keep_prob):
        
        #keep_prob = tf.placeholder(tf.float32)
        
        logits1 = tf.matmul(input, weights1) + bias1
        
        logits2 = tf.nn.tanh(logits1)
        logits3 = tf.matmul(logits2,weights2 ) + bias2
        logits = tf.nn.dropout(logits3 , keep_prob) if keep_prob > 0 else logits3 
        
        return logits
    
       
    
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits_train = get_logits(  tf_train_dataset, weights1, bias1, weights2, bias2, 0.9)
    loss = (tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_train)) + \
    0.01*tf.nn.l2_loss(weights1) + \
    0.01*tf.nn.l2_loss(weights2))
    
    
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits_train)
    valid_prediction = tf.nn.softmax(get_logits(  tf_valid_dataset, weights1, bias1, weights2, bias2,0))
    test_prediction =  tf.nn.softmax(get_logits(  tf_test_dataset, weights1, bias1, weights2, bias2,0))
    
    



In [6]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph4) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  

Initialized


Loss at step 0: 3189.628418
Training accuracy: 9.9%
Validation accuracy: 26.1%


Loss at step 100: 1155.223389
Training accuracy: 80.5%
Validation accuracy: 69.6%


Loss at step 200: 424.138580
Training accuracy: 84.4%
Validation accuracy: 72.7%


Loss at step 300: 156.447937
Training accuracy: 82.2%
Validation accuracy: 77.5%


Loss at step 400: 58.108631
Training accuracy: 81.3%
Validation accuracy: 78.4%


Loss at step 500: 21.697174
Training accuracy: 85.1%
Validation accuracy: 82.2%


Loss at step 600: 8.481359
Training accuracy: 83.5%
Validation accuracy: 83.1%


Loss at step 700: 3.654078
Training accuracy: 81.9%
Validation accuracy: 82.6%


Loss at step 800: 1.884416
Training accuracy: 80.7%
Validation accuracy: 82.5%
Test accuracy: 89.3%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
